In [ ]:
from sklearn.datasets import make_classification
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb

In [ ]:
# load cleaned data
df = pd.read_csv('trash.csv')
df = df.drop('Unnamed: 0', axis=1)

In [ ]:
df = df.sort_values(by=['srch_id'])
replace_dict = dict(zip(df['srch_id'].unique(), list(range(len(df['srch_id'].unique())))))
def replace_search_id(qd):
    return replace_dict[qd['srch_id']]

df["srch_id"] = df.apply(replace_search_id,axis=1)

In [ ]:
def relevance_score_click_bool(qd):
    id = qd
    if qd['booking_bool'] == 1:
        return 5
    if qd['click_bool'] == 1:
        return 1
    return 0

df["relevance"] = df.apply(relevance_score_click_bool,axis=1)

In [ ]:
# # dit is vgm dus niet nodig
# # Function to assign ranking based on book and click status
# def assign_rank(row):
#     if row['booking_bool']:
#         return 1  
#     elif row['click_bool']:
#         return 2  
#     else:
#         return 3  

# df['rank'] = df.apply(assign_rank, axis=1)
# df['rank'] = df.groupby('srch_id')['rank'].rank(method='first')
# df = df.sort_values(by=['srch_id', 'rank'])

In [ ]:
ids = df['srch_id'].unique()
train_idx, validation_idx = train_test_split(ids, test_size=0.2)
train_idx, validation_idx

In [ ]:
train = df[df['srch_id'].isin(train_idx)]
val = df[df['srch_id'].isin(validation_idx)]

In [ ]:
train_rank = train['relevance']
train_qid = train['srch_id']
train_x = train.drop(['click_bool', 'booking_bool', 'relevance','srch_id'],axis=1)

val_rank = val['relevance']
val_qid = val['srch_id']
val_x = val.drop(['click_bool', 'booking_bool', 'relevance','srch_id'],axis=1)

In [ ]:
dtrain = xgb.DMatrix(train_x, label=np.array(train_rank))
groups = train.groupby('srch_id').size().values
dtrain.set_group(groups)


In [ ]:
import numpy as np
import xgboost as xgb

def ndcg_5(preds, dtrain):
    labels = dtrain.get_label()  # True relevance
    preds = np.reshape(preds, (len(labels), -1))  #predicitonss
    num_queries = len(dtrain.get_group())  # number of unique searches

    # for i in range(num_queries):
    #     start_idx = dtrain.get_group()[:i-1].sum() 

    # DCG_p = 0
    # for index in range(5):
    #     rel_i = 

    #     current_ratio=(2**(ordered_data[index])-1)*(math.log((float(index)+2), 2)**(-1))
    #     DCG_p = DCG_p + current_ratio
    # ordered_data.sort(reverse=True)  
    # K = len(ordered_data)
    # indexloop = range(0, K)
    # iDCG_p = 0
    # for index in indexloop:
    #     current_ratio=(2**(ordered_data[index])-1)*((math.log((index+2), 2))**(-1))
    #     iDCG_p = iDCG_p + current_ratio
    # return(DCG_p/iDCG_p)

model = xgb.train({'tree_method': 'hist', 'seed': 1994},  # any other tree method is fine.
           dtrain=dtrain,
           num_boost_round=10,
           obj=ndcg_5)

In [ ]:
# ranker2 = xgb.XGBRanker(tree_method="hist", lambdarank_num_pair_per_sample=8, objective="rank:ndcg", lambdarank_pair_method="topk")
# ranker2.fit(train_x, train_rank, qid = train_qid)